In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
# from pandas.tools.plotting import scatter_matrix
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn import tree
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge,Lasso,BayesianRidge,SGDRegressor,ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import randint

import seaborn as sns

Mount your Google Drive. In this notebook, we assume that 'report1' folder is placed directly under 'My Drive'.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !ls /content/drive/My\ Drive  #You should be able to see 'report1' folder by running this.

In [ ]:
root_dir="./"
N = 20000

train_data = pd.read_csv(root_dir+"train.csv")
test_data = pd.read_csv(root_dir+"test.csv")

train_data = train_data.drop(['url'], axis=1) #remove 'url' information.
train_data = train_data.drop(['timedelta'], axis=1) #remove 'url' information.
train_data

* remove outliers

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] =6.0
fig_size[1] = 6.0
#plt.rcParams["figure.figsize"] = fig_size

x = train_data['shares']
plt.hist(x, normed=True, bins=250)
plt.ylabel('shares');

In [ ]:
u = np.median(train_data['shares'])
s = np.std(train_data['shares'])
line=u+2*s
new_train_data=train_data[train_data['shares']<line]
new_train_data.shape

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] =6.0
fig_size[1] = 6.0
#plt.rcParams["figure.figsize"] = fig_size

x = new_train_data['shares']
plt.hist(x, normed=True, bins=250)
plt.ylabel('shares');

* Standarization

In [ ]:
# minmax_scaler = MinMaxScaler()
# minmax_scaler.fit(new_train_data)
# new_train_data = minmax_scaler.transform(new_train_data)

* Hyperparameter tuning

In [ ]:
# X_train, X_val= train_test_split(new_train_data, test_size=0.2, random_state=42)

# y_train = np.matrix(X_train['shares']).T
# X_train = np.matrix(X_train.drop(['shares'], axis=1)) #Dropping both 'shares', the predicted variable and 'url', a text variable

# y_val = np.matrix(X_val['shares']).T
# X_val = np.matrix(X_val.drop(['shares'], axis=1))

# from sklearn import svm
# from sklearn.model_selection import GridSearchCV, train_test_split

# svrgs_parameters = {
#     'kernel': ['rbf', 'poly'],
#     'C':      [1.0, 2.0, 3.0, 6.0, 10],
#     'gamma':  ['scale', 'auto'],
# }

# svr_cv = GridSearchCV(svm.SVR(), svrgs_parameters, cv=8, scoring= 'neg_mean_squared_error')
# svr_cv.fit(X_train, y_train)
# print("SVR GridSearch score: "+ str(svr_cv.best_score_))
# print("SVR GridSearch params: " + str(svr_cv.best_params_))

In [ ]:
# model_svr_best = SVR(kernel='rbf', C=6, gamma='auto')
# model_svr_best.fit(X_train, y_train)

In [ ]:
# final_prediction = model_svr_best.predict(X_val)
# def MABS(vec1, vec2):
#     return np.mean(np.abs(vec1 - vec2))

# # reg = Lasso(alpha=10.0).fit(XTrain, yTrain)   #alpha: regularization strength
# yHatTrain = model_svr_best.predict(X_train)
# yHatVal = model_svr_best.predict(X_val)
# print("Training error ", MABS(y_train, yHatTrain.T))
# print("Validation error ", MABS(y_val, yHatVal.T))

In [ ]:
# x = pd.get_dummies(new_train_data.iloc[:,1:59])
# # MEDVとの相関性
# plt.figure(figsize=(10,7))
# x.corr()['shares'].sort_values(ascending = False).plot(kind='bar')
# plt.title("Correlations between MEDV and variables")

* Correlations

In [ ]:
c = new_train_data.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")
so = pd.DataFrame(data=so, columns=['Pearson Coeff'])
so = so[(so['Pearson Coeff'] >= 0.8) & (so['Pearson Coeff'] < 1.0)]
so.iloc[::2]

In [ ]:
sns.set(style="white")
f = plt.figure(figsize = (10,10))
sns.heatmap(new_train_data.corr(),cmap=sns.diverging_palette(220, 10, as_cmap=True),vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},mask=np.triu(np.ones_like(new_train_data.corr(), dtype=np.bool)))

* GridSearch

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
N=200

# X = np.matrix(train_data.drop(['shares'], axis=1)) 
# y = np.matrix(train_data['shares']) #This is the target
X = new_train_data
train_df = X.sample(frac=0.99, random_state = None)
yTrain = train_df['shares']
XTrain = train_df.drop(['shares'], axis=1) #use the first N samples for training
# print(XTrain.shape,yTrain.shape)

possible_features = new_train_data.columns.copy().drop('shares')
# print(possible_features)
selector = SelectKBest(f_regression, len(possible_features))
selector.fit(XTrain[possible_features], yTrain)
scores = -np.log10(selector.pvalues_)
indices = np.argsort(scores)[::-1]
# print('Feature importances:')
topfeatures=[]
for i in range(20):
    # print('%.2f %s' % (scores[indices[i]], possible_features[indices[i]]))
    topfeatures.append(possible_features[indices[i]])
print(topfeatures)

In [ ]:
# corr = new_train_data.corr()
# flag = np. full((corr.shape[0]),True, dtype=bool)
# for i in range(corr. shape[0]):
#     for j in range(i+1, corr. shape[0]):
#         if corr.iloc[i,j] >= 0.6:
#             if flag[j]:
#                 flag[j] = False
# select = new_train_data.columns [flag]. tolist()
# new_train_data = new_train_data[select]
# select.remove( 'shares')
# test_data = test_data[select]
# print(new_train_data.columns)

In [ ]:
selected_columns=['self_reference_avg_sharess','self_reference_max_shares','self_reference_min_shares',
'global_sentiment_polarity','global_subjectivity',
'kw_max_avg','kw_min_avg','kw_avg_max','kw_avg_avg',
'num_hrefs', 'num_imgs','num_videos','num_keywords',
'LDA_00','LDA_01','LDA_02','LDA_03',
'weekday_is_saturday','weekday_is_sunday', 'is_weekend',
'avg_negative_polarity',
'rate_negative_words',
'data_channel_is_entertainment', 'data_channel_is_tech','data_channel_is_world','data_channel_is_socmed']
# selected_columns=topfeatures
# selected_columns.append('shares')
# new_train_data=new_train_data[selected_columns]
# selected_columns
# len(selected_columns)

* Features recomposition

In [ ]:
date_features=['weekday_is_saturday','weekday_is_sunday', 'is_weekend']
# ref_features=['self_reference_avg_sharess','self_reference_max_shares','self_reference_min_shares']
# topic2_features=['LDA_02','data_channel_is_world']
# senti_features=['global_sentiment_polarity','global_subjectivity']
dele_features=[date_features]#date_features,ref_features,topic2_features,senti_features
recomp_features=['date']#'date','ref','topic2','senti'

for dele_ind in range(len(dele_features)):
    recomp_feature=recomp_features[dele_ind]
    new_train_data[recomp_feature]=0
    test_data[recomp_feature]=0
    for dele_feature in dele_features[dele_ind]:
        new_train_data[recomp_feature]+=new_train_data[dele_feature]
        test_data[recomp_feature]+=test_data[dele_feature]
        selected_columns.remove(dele_feature)
    mask=(new_train_data[recomp_feature]>0)
    new_train_data[recomp_feature][mask]=1
    new_train_data[recomp_feature][~mask]=0
    mask=(test_data[recomp_feature]>0)
    test_data[recomp_feature][mask]=1
    test_data[recomp_feature][~mask]=0
    selected_columns.append(recomp_feature)

test_data=test_data[selected_columns]
selected_columns
selected_columns.append('shares')
new_train_data=new_train_data[selected_columns]
selected_columns

* Split data

In [ ]:
target = new_train_data['shares']
new_train_data=new_train_data.drop(['shares'], axis=1)

minmax_scaler = StandardScaler()
minmax_scaler.fit(new_train_data)
new_train_data = minmax_scaler.transform(new_train_data)

XTrain, XVal, yTrain, yVal = train_test_split(new_train_data, target, test_size=0.2, random_state=42)

yTrain = np.float32(yTrain).T
XTrain = np.float32(XTrain) #Dropping both 'shares', the predicted variable and 'url', a text variable

yVal = np.float32(yVal).T
XVal = np.float32(XVal)

minmax_scaler.fit(test_data)
test_data = minmax_scaler.transform(test_data)
XTest = np.float32(test_data) #final testing data
print(XTrain.shape,yTrain.shape)

In [ ]:
# from sklearn.pipeline import Pipeline
# pipe_svr = Pipeline([("StandardScaler",StandardScaler()),
#                      ("svr",SVR())])
# param_range = [1000.0,2000.0,5000.0,10000.0]
# param_grid = [{"svr__C":param_range,"svr__kernel":["linear"]}]
#               # 注意__是指两个下划线，一个下划线会报错的
#             #   {"svr__C":param_range,"svr__gamma":param_range,"svr__kernel":["rbf"]}]
# gs = GridSearchCV(estimator=pipe_svr,
#                   param_grid = param_grid,
#                   scoring = 'r2',
#                   cv = 10)       # 10折交叉验证
# gs = gs.fit(XTrain,yTrain)

In [ ]:
# print("网格搜索最优得分：",gs.best_score_)
# print("网格搜索最优参数组合：\n",gs.best_params_)

In [ ]:
# svrgs_parameters = {
#     # 'kernel': ['rbf', 'poly'],
#     'C':      [1.0, 10.0, 100.0, 1000.0],
#     # 'gamma':  ['scale', 'auto'],
# }

# svr_cv = GridSearchCV(SVR(), svrgs_parameters, cv=8, scoring= 'neg_mean_squared_error')
# svr_cv.fit(XTrain, yTrain)
# print("SVR GridSearch score: "+ str(svr_cv.best_score_))
# print("SVR GridSearch params: " + str(svr_cv.best_params_))

* Cross_validation

In [ ]:
# estimator = SVR(kernel="linear")
# selector = RFECV(estimator, step=1, cv=3)
# rfecv = selector.fit(XTrain, yTrain)

# print("Optimal number of features: %d" % rfecv.n_features_)
# print('Selected features: %s' % list(X.columns[rfecv.support_]))

# plt.figure(figsize=(10, 6))
# plt.xlabel("Number of features selected")
# plt.ylabel("Cross validation score (no. of correct classifications)")
# plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
# plt.show()

In [ ]:
def MABS(vec1, vec2):
    return np.mean(np.abs(vec1 - vec2))

model=SVR(C=310)
model.fit(XTrain, yTrain)
yHatTrain = model.predict(XTrain)
yHatVal = model.predict(XVal)
print("Training error ", MABS(yTrain, yHatTrain.T))
print("Validation error ", MABS(yVal, yHatVal.T))

In [ ]:
yHatTest = model.predict(XTest)
np.savetxt(root_dir+'result.txt', yHatTest) #save predictions in rows

* Random Forest

In [ ]:
model = RandomForestRegressor(random_state=42)
model.fit(XTrain, yTrain)
yHatTrain = model.predict(XTrain)
yHatVal = model.predict(XVal)
print("Training error ", MABS(yTrain, yHatTrain.T))
print("Validation error ", MABS(yVal, yHatVal.T))

* Decision Tree

In [ ]:
model = DecisionTreeRegressor(random_state=42)
model.fit(XTrain, yTrain)
yHatTrain = model.predict(XTrain)
yHatVal = model.predict(XVal)
print("Training error ", MABS(yTrain, yHatTrain.T))
print("Validation error ", MABS(yVal, yHatVal.T))

* GradientRegression 2151

In [ ]:
model = GradientBoostingRegressor(random_state=42)
model.fit(XTrain, yTrain)
yHatTrain = model.predict(XTrain)
yHatVal = model.predict(XVal)
print("Training error ", MABS(yTrain, yHatTrain.T))
print("Validation error ", MABS(yVal, yHatVal.T))

* Bayes 2142

In [ ]:
model = BayesianRidge()
model.fit(XTrain, yTrain)
yHatTrain = model.predict(XTrain)
yHatVal = model.predict(XVal)
print("Training error ", MABS(yTrain, yHatTrain.T))
print("Validation error ", MABS(yVal, yHatVal.T))

* SVR Feature selection

In [ ]:
# SVR_sel = SVR(kernel="linear")  //linear效果不行不如rbf，但是rbf不支持特征挑选
# selector=RFECV(SVR_sel,step=1,cv=5)
# selector.fit(XTrain, yTrain)
# selector.ranking_

In [ ]:
# XTrain=XTrain[selector.support_]
# yTrain=yTrain[selector.support_]
# XTrain=XTest[selector.support_]
# SVR_reg = SVR()
# SVR_reg.fit(XTrain, yTrain)

* SVR

In [ ]:
# SVR_reg = SVR()
# SVR_reg.fit(XTrain, yTrain)

Run Ridge regression.

In [ ]:
model = Lasso(alpha=10.0).fit(XTrain, yTrain)   #alpha: regularization strength
yHatTrain = model.predict(XTrain)
yHatVal = model.predict(XVal)
print("Training error ", MABS(yTrain, yHatTrain.T))
print("Validation error ", MABS(yVal, yHatVal.T))

Evaluate training and validation errors.

In [ ]:
# create dataset for lightgbm
# lgb_train = lgb.Dataset(XTrain, yTrain)
# lgb_eval = lgb.Dataset(XVal, yVal, reference=lgb_train)

# specify your configurations as a dict
# params = {
# 'boosting_type': 'gbdt',
# 'objective':'mae',
# 'n_jobs':8,
# 'subsample': 0.5,
# 'subsample_freq': 1,
# 'learning_rate': 0.01,
# 'num_leaves': 2**11-1,
# 'min_data_in_leaf': 2**12-1,
# 'feature_fraction': 0.5,
# 'max_bin': 100,
# 'n_estimators': 2500,
# 'boost_from_average': False,
# "random_seed":1,
# }

# print('Starting training...')
# # train
# verbose=100
# model = LGBMRegressor(**params)
# model.fit(XTrain,yTrain,
#                 eval_set=[(XVal,yVal)],
#                 early_stopping_rounds=verbose,
#                 verbose=verbose)
# # with open(f'model_{num}.pkl', 'wb') as handle:
# #     pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

# def MABS(vec1, vec2):
#     return np.mean(np.abs(vec1 - vec2))

# yHatTrain = model.predict(XTrain)
# yHatVal = model.predict(XVal)
# print("Training error ", MABS(yTrain, yHatTrain.T))
# print("Validation error ", MABS(yVal, yHatVal.T))

# yHatTest=model.predict(XTest)
# np.savetxt(root_dir+'result.txt', yHatTest) #save predictions in rows

# params,
#                 lgb_train,
#                 num_boost_round=20,
#                 valid_sets=lgb_eval,
#                 early_stopping_rounds=5)

# print('Saving model...')
# # save model to file
# gbm.save_model('model.txt')

# print('Starting predicting...')
# # predict
# y_pred = gbm.predict(XTest, num_iteration=gbm.best_iteration)
# eval
# print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)



Now, project the testing data. You can upload 'result.txt' to the evaluation server.

In [ ]:
yHatTest = model.predict(XTest)
np.savetxt(root_dir+'result.txt', yHatTest) #save predictions in rows